In [1]:
import numpy as np
import pyvista as pv
import torch
import sys

In [2]:
# 定义字节到 MB 和 GB 的转换函数
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB")
    i = int((size_bytes).bit_length() // 10)  # 根据大小选择合适的单位
    p = 1024 ** i
    size = round(size_bytes / p, 2)
    return f"{size} {size_name[i]}"

In [12]:
film = np.zeros((100, 100, 2000, 3))

bottom = 1900
height = 1940

density = 10

sphere = np.ones((100, 100, 2000), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom
# film[:, :30, bottom:height, 1] = 10
# film[:, -30:, bottom:height, 1] = 10


In [11]:
depo1 = torch.Tensor(film[:, :50, bottom-200:bottom+50, 2]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :50, bottom-200:bottom+50,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:57295/index.html?ui=P_0x1a98db51e50_1&reconnect=auto" class="pyvis…

In [16]:
film = np.zeros((100, 100, 200, 3))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 200), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom

In [12]:
size_in_bytes = sys.getsizeof(film)  # 获取对象大小
print(f"Size of variable: {convert_size(size_in_bytes)}")

Size of variable: 30.52 MB


In [13]:
etchfilm = film

In [5]:
import Multi_Species_bosch_etching_noMirrorSymmtry

In [14]:
logname = 'Multi_species_benchmark_0919'
testEtch = Multi_Species_bosch_etching_noMirrorSymmtry.etching(inputMethod='bunch', depo_or_etching='etching', 
                etchingPoint = np.array([center, center, 37]),depoPoint = np.array([center, center, 37]),
                density=density, center_with_direction=np.array([[35,100,75]]), 
                range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                reaction_type=False, param = [1.6, -0.7],
                sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                celllength=1e-5, kdtreeN=5, tstep=1e-5,
                substrateTop=bottom,posGeneratorType='top', logname=logname)

In [15]:
cicle = 100

for i in range(cicle):
    step1 = testEtch.inputParticle('maxwell', 0, int(2e4),int(3e7), 10)
    np.save('./bosch_data_0919/bosch_sf_step_{}_sf'.format(i), etchfilm)
    step2 = testEtch.inputParticle('maxwell', 1, int(2e4),int(1e7), 10)
    indice_c4 = etchfilm[:, :, :, 1] >= 10
    etchfilm[indice_c4, 1] = 10
    np.save('./bosch_data_0919/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    step3 = testEtch.inputParticle('updown', 2, int(2e4),int(5e6), 10)
    np.save('./bosch_data_0919/bosch_sf_step_{}_Ar'.format(i), etchfilm)

running: 100%|██████████████████████████████████████████████████████| 100/100 [10:24<00:00, 6.25s/B]
running:  88%|██████████████████████████████████████████████▋      | 88.0/100 [59:23<08:05, 40.5s/B]


KeyboardInterrupt: 

In [13]:
np.save('bosch_sf_step11', etchfilm)

In [24]:
etching = np.load('./bosch_data/bosch_sf_step_0_sf.npy')

In [29]:
etching = np.load('./bosch_data/bosch_sf_step_0_c4f8.npy')

In [27]:
etching = np.load('./bosch_data/bosch_sf_step_0_Ar.npy')

In [25]:
depo1 = torch.Tensor(etching[:, :, bottom-50:, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-50:,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:49836/index.html?ui=P_0x1e5a0fc0e60_12&reconnect=auto" class="pyvi…

In [30]:
depo1 = torch.Tensor(etching[:, :50, bottom-200:bottom+50, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :50, bottom-200:bottom+50,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:49836/index.html?ui=P_0x1e525d87440_15&reconnect=auto" class="pyvi…

In [13]:
depo1 = torch.Tensor(etching[:, :, bottom-400:bottom-200, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-400:bottom-200,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:49836/index.html?ui=P_0x1e506ac6660_3&reconnect=auto" class="pyvis…

In [14]:
depo1 = torch.Tensor(etching[:, :75, bottom-400:bottom-200, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :75, bottom-400:bottom-200,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:49836/index.html?ui=P_0x1e5032db890_4&reconnect=auto" class="pyvis…